This code uses the motion-cosegmentation github repo and its trained model to change the face of a driver video to be the face of from an image. 
It first uses ffmpeg to crop the face out of the driver video, then uses the part swap module to apply the face from the image onto the face from the video. 
Lastly it reapplys the cropped out video back onto the original location in the driver video.

In [ ]:
# Face swap model
!gdown --id 1XmBc450Ho-bExSlJM3Ii6NTn4kQniGZR

Downloading...
From: https://drive.google.com/uc?id=1XmBc450Ho-bExSlJM3Ii6NTn4kQniGZR
To: /content/vox-10segments.pth.tar
751MB [00:07, 95.4MB/s]


In [ ]:
# Face image
!gdown --id 1AyUnjSoHAOomJs2X8xD_JLkc8JVpSGJd

Downloading...
From: https://drive.google.com/uc?id=1AyUnjSoHAOomJs2X8xD_JLkc8JVpSGJd
To: /content/guysmiling_cropped.jpg
100% 340k/340k [00:00<00:00, 45.7MB/s]


In [ ]:
# Source video
!gdown --id 1c59IkePQ-n6tFwsrEyDQjXCPBMIPhiGp

Downloading...
From: https://drive.google.com/uc?id=1c59IkePQ-n6tFwsrEyDQjXCPBMIPhiGp
To: /content/lowres.mp4
2.85MB [00:00, 90.6MB/s]


In [ ]:
# Crop out face
!ffmpeg -i lowres.mp4 -filter:v "crop=256:256:120:20" cropped.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
!git clone https://github.com/AliaksandrSiarohin/motion-cosegmentation motion-co-seg

Cloning into 'motion-co-seg'...
remote: Enumerating objects: 97, done.
remote: Total 97 (delta 0), reused 0 (delta 0), pack-reused 97
Unpacking objects: 100% (97/97), done.


In [ ]:
cd motion-co-seg/

/content/motion-co-seg


In [ ]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
from part_swap import load_checkpoints
from part_swap import make_video
from skimage import img_as_ubyte

warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/guysmiling_cropped.jpg')
target_video = imageio.mimread('/content/cropped.mp4', memtest=False)

source_image = resize(source_image, (256, 256))[..., :3]
target_video = [resize(frame, (256, 256))[..., :3] for frame in target_video]

reconstruction_module, segmentation_module = load_checkpoints(config='config/vox-256-sem-10segments.yaml', checkpoint='/content/vox-10segments.pth.tar', blend_scale=1)

predictions = make_video(swap_index=[2,3,5,9], source_image = source_image, target_video = target_video,
                             segmentation_module=segmentation_module, reconstruction_module=reconstruction_module)

imageio.mimsave('/content/generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=29.97)


100%|██████████| 741/741 [00:31<00:00, 23.30it/s]


In [ ]:
!ffmpeg -i ../lowres.mp4 -vf "movie=../generated.mp4, scale=256:256 [inner];[in][inner] overlay=120:20[out]" ../final.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [ ]:
# Show final video
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/final.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)